In [1]:
import pandas as pd
import numpy as np
from io import BytesIO
import requests
from urllib.parse import urlencode
import urllib
from requests.auth import HTTPBasicAuth
from requests.exceptions import ChunkedEncodingError
import os
import json
import yadisk
from datetime import datetime, date, timedelta
import locale
from time import sleep
import shutil
import gc
import turbodbc
from turbodbc import connect
import gc
from pandas.api.types import is_string_dtype
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import warnings
import re

import config
from yandex_disk_func import *
from parse_functions import *



# # указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\35_mediaplan_parser'

public_key = config.public_key # обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета

# забираем список папок в нужной нам директории
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
pd.options.mode.chained_assignment = None

In [3]:
base_cols = ['supplier', 'report_name', 'sheet_name', 'brand', 'period', 'source', 'site/ssp', 'placement', 
                     'targeting', 'geo', 'soc_dem', 'ad copy format', 'rotation type', 
                     'unit price', 'frequency', 'reach', 'impressions', 'clicks', 
             'budget_without_nds', 'budget_nds', 'views', 'vtr, %']

In [4]:
public_key = yandex_folders['public_key']  # из ответа Яндекс забираем public_key, чтобы использовать его для скачивания файлов

for i in range(len(yandex_folders['_embedded']['items'])): # через цикл проходим по ответу Яндекса и забираем названия вложенных папок
    file_type = yandex_folders['_embedded']['items'][i]['type']
    if file_type=='dir':   # если находим файлы с типом dir (папка), то забираем путь к этой папке
        folder_path = yandex_folders['_embedded']['items'][i]['path']
        print(folder_path)
        yandex_responce = get_yandex_disk_responce(base_public_url, public_key, folder_path) # отправляем запрос, чтобы получить содержимое папки

        # Через цикл проходим по папке с файлами
        # Нас интересуют файлы эксель. Причем каждая экселька будет парситься по своему, т.к. они относятся к разным рекламным площадкам
        
        # Проходим через цикл по содержимому папки (отдельный флайт)
        for i in range(len(yandex_responce['_embedded']['items'])):
            file_info = yandex_responce['_embedded']['items'][i]
            if file_info['type']=='file':  # если документ является фалйом(не папкой или изображением), то забираем его название 
                file_name = file_info['name'] # сохраняем название файла
                if 'xls' in file_name: # еслит тип файла является xlsx, то уберем расширение и будем его использовать в качесвте названия отчета
                    file_path = file_info['path']
                    
                    report_name = '.'.join(file_name.split('.')[:-1]) # убираем .xlsx из названия файла
                    print(report_name)
                    res_file_link = get_yandex_disk_responce(download_url, public_key, file_path) # получаем ссылку на скачивание отчета
                    download_response = requests.get(res_file_link['href'])

/01_yandex
/02_prog
MP_Roxot_Игроник_PSB_Brand-booster_sep-dec24
/03_Полина


In [5]:
data_link = download_response.content

In [6]:
sheet_names = pd.ExcelFile(BytesIO(data_link))

In [7]:
sheet_names.sheet_names

['ПСБ BB Banner',
 'ПСБ BB OLV',
 'Список сайтов OLV',
 'Список сайтов баннеры',
 'Тех. требования к материалам',
 'Технические требования к кликов']

In [86]:
network = 'roxot'
report_name = 'tst'
tmp_dict = {}
sheet_names = pd.ExcelFile(BytesIO(data_link))
for sheet_name in sheet_names.sheet_names:
    if 'псб bb banner' in sheet_name.lower():
        df = pd.read_excel(BytesIO(data_link), sheet_name=sheet_name, header=None)
        sheet_name = normalize_headers(sheet_name)
        print(sheet_name)
        # заголовки в файле состоят из 2-х строк, поэтому нужно выполнить заполнение вниз на 1 строку
        # забираем название полей, в которых нужно сдвинуть строку вниз
        ffill_columns = [1, 2, 3, 4, 5, 6, 7]
        df[ffill_columns] = df[ffill_columns].ffill(limit=1) # заполняем вниз
        df[8] = df[8].fillna('0')
        df = df.fillna('')
        # сохраняем название бренда
        brand = df[2].loc[get_index_row(df, 1, 'клиент')]
        
        # т.к. мы выполнили заполнение вниз на 1 строку
        # у нас дублируются заголовки, поэтому мы берем второе вхождение забираем индекс начала таблицы
        start_index = get_index_row(df, 1, 'аудитория') + 1
        # задаем названия полей
        df.columns = df.iloc[start_index].apply(normalize_headers) # забираем название полей из файла
        # обрезаем верхнюю часть таблицы. она больше не нужна
        df = df.iloc[start_index+4:].reset_index(drop=True)
        # обрезаем таблицу снизу
        end_index = get_index_row(df, 'частота', '0')
        df = df.iloc[:end_index]
        # создаем базовый список полей, которые есть всегда вне зависимости от типа размещения
        standart_columns = ['медиаканал', 'форматы объявлений', 'инвентарь', 'период', 'таргетинг', 'частота', 
                        'охват, прогноз', 'показы', 'клики, прогноз', 'бюджет без ндс', 'avg. cpm']
        # проверяем наличие Видео размещений. Если они есть, то используем дополнительные поля из таблицы
        # если Видео размещений нет, то добавляем дополнительно 2 поля с 0 (это нужно для нормализации общей таблицы     
        if 'vtr, %' not in list(df.columns):
            df['views'] = 0
            df['vtr, %'] = 0.0
        
        standart_columns += ['views', 'vtr, %']
        # оставляем только нужные поля
        df = df[standart_columns]
         # приводим названия полей к единому стандарту
        df = df.rename(columns={'медиаканал': 'source', 'форматы объявлений': 'ad copy format', 'инвентарь': 'placement', 'период': 'period',
                                'таргетинг': 'targeting', 'частота': 'frequency', 'охват, прогноз': 'reach', 'показы': 'impressions',
                                'клики, прогноз': 'clicks', 'avg. cpm': 'unit price', 'бюджет без ндс': 'budget_without_nds'})
        df['supplier'] = network
        df['report_name'] = report_name
        df['sheet_name'] = sheet_name
        df['brand'] = brand
        df['site/ssp'] = ''
        df['rotation type'] = 'CPM'
        df['budget_nds'] =(df['budget_without_nds'] * 1.2).astype('float').round(2)
        df['geo'] = df['targeting'].apply(lambda x: get_targetings(x, 'гео:','\n', flag='geo'))
        df['soc_dem'] = df['targeting'].apply(lambda x: get_targetings(x, 'соц.дем:','\n', flag='soc'))
        df['reach'] = df['reach'].astype('int')
        df['clicks'] = df['clicks'].astype('int')
        df = df[base_cols]

псб bb banner


In [87]:
df

14,supplier,report_name,sheet_name,brand,period,source,site/ssp,placement,targeting,geo,soc_dem,ad copy format,rotation type,unit price,frequency,reach,impressions,impressions,clicks,budget_without_nds,budget_nds,views,"vtr, %"
0,roxot,tst,псб bb banner,ПСБ Имидж Спецпроект (осень-зима 2024),01.09.24 - 30.09.24,Roxot Premium Display Network,,WL/премиум инвентарь Roxot,"Гео: РФ\nСоц.дем: 25+\nGPT таргетинг: МСБ: (предприниматели, бизнес, услуги банков, РКО, инвестиции, ВЭД)",рф,25+,Brand-booster Banners,CPM,300,2,833333,1666667,1666667,13333,500000.1,600000.12,0,0.0
1,roxot,tst,псб bb banner,ПСБ Имидж Спецпроект (осень-зима 2024),01.10.24 -31.10.24,Roxot Premium Display Network,,WL/премиум инвентарь Roxot,"Гео: РФ\nСоц.дем: 25+\nGPT таргетинг: МСБ: (предприниматели, бизнес, услуги банков, РКО, инвестиции, ВЭД)",рф,25+,Brand-booster Banners,CPM,300,2,833333,1666667,1666667,13333,500000.1,600000.12,0,0.0
2,roxot,tst,псб bb banner,ПСБ Имидж Спецпроект (осень-зима 2024),01.11.24 - 30.11.24,Roxot Premium Display Network,,WL/премиум инвентарь Roxot,"Гео: РФ\nСоц.дем: 25+\nGPT таргетинг: МСБ: (предприниматели, бизнес, услуги банков, РКО, инвестиции, ВЭД)",рф,25+,Brand-booster Banners,CPM,300,2,833333,1666667,1666667,13333,500000.1,600000.12,0,0.0
3,roxot,tst,псб bb banner,ПСБ Имидж Спецпроект (осень-зима 2024),01.12.24 - 31.12.24,Roxot Premium Display Network,,WL/премиум инвентарь Roxot,"Гео: РФ\nСоц.дем: 25+\nGPT таргетинг: МСБ: (предприниматели, бизнес, услуги банков, РКО, инвестиции, ВЭД)",рф,25+,Brand-booster Banners,CPM,300,2,416666,833333,833333,6666,249999.9,299999.88,0,0.0
4,roxot,tst,псб bb banner,ПСБ Имидж Спецпроект (осень-зима 2024),01.09.24 - 30.09.24,Roxot Premium Display Network,,WL/премиум инвентарь Roxot,"Гео: РФ\nСоц.дем: 25+\nGPT таргетинг: РБ: (услуги банков, кредитные карты, кредиты, дебетовые карты, вклады, инвестиции)",рф,25+,Brand-booster Banners,CPM,300,2,833333,1666667,1666667,13333,500000.1,600000.12,0,0.0
5,roxot,tst,псб bb banner,ПСБ Имидж Спецпроект (осень-зима 2024),01.10.24 -31.10.24,Roxot Premium Display Network,,WL/премиум инвентарь Roxot,"Гео: РФ\nСоц.дем: 25+\nGPT таргетинг: РБ: (услуги банков, кредитные карты, кредиты, дебетовые карты, вклады, инвестиции)",рф,25+,Brand-booster Banners,CPM,300,2,833333,1666667,1666667,13333,500000.1,600000.12,0,0.0
6,roxot,tst,псб bb banner,ПСБ Имидж Спецпроект (осень-зима 2024),01.11.24 - 30.11.24,Roxot Premium Display Network,,WL/премиум инвентарь Roxot,"Гео: РФ\nСоц.дем: 25+\nGPT таргетинг: РБ: (услуги банков, кредитные карты, кредиты, дебетовые карты, вклады, инвестиции)",рф,25+,Brand-booster Banners,CPM,300,2,833333,1666667,1666667,13333,500000.1,600000.12,0,0.0
7,roxot,tst,псб bb banner,ПСБ Имидж Спецпроект (осень-зима 2024),01.12.24 - 31.12.24,Roxot Premium Display Network,,WL/премиум инвентарь Roxot,"Гео: РФ\nСоц.дем: 25+\nGPT таргетинг: РБ: (услуги банков, кредитные карты, кредиты, дебетовые карты, вклады, инвестиции)",рф,25+,Brand-booster Banners,CPM,300,2,416666,833333,833333,6666,249999.9,299999.88,0,0.0


In [48]:
df.columns

Index(['', 'целевая аудитория', 'медиаканал', '', 'форматы объявлений',
       'инвентарь', 'период', 'таргетинг', 'частота', 'охват, прогноз',
       'показы', 'клики, прогноз', 'ctr, прогноз', 'avg. cpc', 'avg. cpm',
       'бюджет без ндс'],
      dtype='object', name=14)

14

In [ ]:
network = 'mobidriven'
report_name = 'tst'
tmp_dict = {}
sheet_names = pd.ExcelFile(BytesIO(data_link))
for sheet_name in sheet_names.sheet_names:
    if 'мп' in sheet_name.lower():
        
        df = pd.read_excel(BytesIO(data_link), sheet_name=sheet_name, header=None)
        sheet_name = normalize_headers(sheet_name)
        print(sheet_name)
        df[0] = df[0].fillna('0')
        df = df.fillna('')
        

        # сохраняем название бренда
        brand = df[1].loc[get_index_row(df, 0, 'клиент')]
        # сохраняем период
        period = df[1].loc[get_index_row(df, 0, 'период')]
        # сохраняем ЦА
        soc_dem = df[1].loc[get_index_row(df, 0, 'ца')]
       
        # забираем индекс начала таблицы
        start_index = get_index_row(df, 0, 'категория')
        # задаем названия полей
        df.columns = df.iloc[start_index].apply(normalize_headers) # забираем название полей из файла
        # обрезаем верхнюю часть таблицы. она больше не нужна
        df = df.iloc[start_index+2:].reset_index(drop=True)
        # обрезаем таблицу снизу
        end_index = get_index_row(df, 'категория / инструмент', '0')
        df = df.iloc[:end_index]
        
        df['формат'] = df.apply(get_timing, axis=1)

        # создаем базовый список полей, которые есть всегда вне зависимости от типа размещения
        standart_columns = ['категория / инструмент', 'таргетинги / сегменты', 'тип трафика', 'формат', 'гео', 'единица закупки', 
                        'цена за единицу', 'стоимость, руб, без учета ндс, 20%',
                        'показы', 'частота', 'охват', 'клики']
        # проверяем наличие Видео размещений. Если они есть, то используем дополнительные поля из таблицы
        # если Видео размещений нет, то добавляем дополнительно 2 поля с 0 (это нужно для нормализации общей таблицы     
        if 'vtr, %' not in list(df.columns):
            df['views'] = 0
            df['vtr, %'] = 0.0
        
        standart_columns += ['views', 'vtr, %']
        # оставляем только нужные поля
        df = df[standart_columns]
            
        # приводим названия полей к единому стандарту
        df = df.rename(columns={'категория / инструмент': 'source', 'тип трафика': 'placement', 'таргетинги / сегменты': 'targeting',
                                'формат': 'ad copy format', 'гео': 'geo', 'единица закупки': 'rotation type',
                       'цена за единицу': 'unit price', 'стоимость, руб, без учета ндс, 20%': 'budget_without_nds',
                       'показы': 'impressions', 'частота': 'frequency', 'охват': 'reach', 'клики': 'clicks', 'досмотры': 'views', 'vtr %': 'vtr, %'})
        df['supplier'] = network
        df['report_name'] = report_name
        df['sheet_name'] = sheet_name
        df['brand'] = brand
        df['period'] = period
        df['site/ssp'] = ''
        df['budget_nds'] =(df['budget_without_nds'] * 1.2).astype('float').round(2)
        # вызываем функцию для парсинга текста из поля targeting
        # значение каждого таргетинга записываем в отдельное поле датаФрейма
        df['soc_dem'] = soc_dem
        # df['views'] = df['views'].apply(normalize_digits)
        # df['vtr, %'] = df['vtr, %'].apply(normalize_digits)
        # если в этих полях встречаются пустые ячейки, то заменяем их на 0
        df['vtr, %'] = df['vtr, %'].apply(replace_blank)
        df['views'] = df['views'].apply(replace_blank)
        df = df[base_cols]

In [ ]:
# создаем функцию, чтобы получить индекс строки с первым встречанием заданного паттерна
def get_index_row(df, col_name, flag):
    return list(df[df[col_name].str.lower().str.contains(flag)].index)[0] 

In [ ]:
# создаем функцию, чтобы забрать хронометраж ролика, если он встречается в ячейке
def get_timing(row):
    result = str(row['формат'])
    timing = str(row['хронометраж ролика'])
    if len(timing) > 0:
        result += f' timing: {timing}'
    return result

In [ ]:
df

In [56]:
text = """Гео: РФ
Соц.дем: 25+
GPT таргетинг: МСБ: (предприниматели, бизнес, услуги банков, РКО,  инвестиции, ВЭД)
"""

In [71]:
text

'гео: рф\nсоц.дем: 25+\ngpt таргетинг: мсб: (предприниматели, бизнес, услуги банков, рко,  инвестиции, вэд)'

In [60]:
text = text.lower().strip()
if 'гео:' in text:
    geo = get_target_text('гео:', '\n', text)
if 'соц:' in text:
    soc_dem = get_target_text('соц:', '\n', text)


In [68]:

geo

'рф'

In [72]:
soc_dem = get_targetings(text, 'соц.дем:','\n', flag='soc')
soc_dem

'25+'

In [67]:
# создаем функцию для записи названий таргетингов в поля датаФрейма
# на входе она принимает 2 параметра
# column - поле с текстом, который нужно распарсить
# source - название источника (какому поставщику принадлежит таблица - таким образом мы понимаем, какие правила парсинга применяются
# для каждого таргетинга отдельно вызываем функцию, чтобы достать нужный текст
# на выходе возвращаем список с текстом для каждого таргетинга
# если таргеитнга не было в тексте, то вернутся пустые строки
def get_targetings(column, start_pattern, end_pattern, flag):
    result = ''
    text = column.lower().strip()
    if 'гео:' in text or 'ца:' in text or 'соц' in text:
        if flag == 'geo':
            result = get_target_text(start_pattern, end_pattern, text)
        if flag == 'soc':
            result = get_target_text(start_pattern, end_pattern, text)
            
    return result

In [ ]:
# создаем функцию для парсинга таргетингов из текста
# на вход она принимает ключевое слово - start_pattern -какой именно таргетинг мы ищем гео: / ца: / интересы:
# окончанием строки - end_pattern - передаем слово или символ, который будет считаться окончанием строки например -  \n или слово покупатели
# если таргетинг находится в конце текста, то возвращаем текст до конца
# на выходе функция текст таргетинга БЕЗ ключевого слова
def get_target_text(start_pattern, end_pattern, text):
    start_index = text.index(start_pattern)
    text = text[start_index:]
    end_index = text.index(end_pattern)
    target_text = text[len(start_pattern):end_index]

    target_text = target_text.strip().replace('\n', ' ')

    return target_text

In [ ]:
geo

In [ ]:
ca

In [ ]:
interests

In [ ]:
start_index = text.index('ца')

In [ ]:
start_index 

In [ ]:
end_index = text[start_index:].index('\n')

In [ ]:
end_index

In [ ]:
target_text = text[8:18]

In [ ]:
target_text

In [ ]:
text